In [1]:
import pandas as pd
import json
path = '/nfs/public/rw/homes/uni_adm/tmp/log_parsing/offset/merged.json'

def read_json_file(json_file):
    with open(json_file) as f:
        return json.load(f)
    
offset_counts = read_json_file(path)
offset_counts['-25'] = offset_counts['not_specified']
del offset_counts['not_specified']
list_offset_counts = [{'offset':int(offset), 'count': int(count)} for offset, count in offset_counts.items()]
df = pd.DataFrame(list_offset_counts)
df = df[df['offset'] % 25 == 0]
df = df.sort_values('offset', ascending=True)
df['%queries'] = 100*df['count']/sum(df['count'])
threshold = 300
significant = df['offset'] <= threshold
other = df['offset'] > threshold
df_significant = df[significant].copy()
df_other = pd.DataFrame([{
    'count':sum(df.loc[other, 'count']),
    'offset': threshold + 25,
    '%queries': sum(df.loc[other, '%queries']),
}])
df_summary = pd.concat([df_significant, df_other], sort=False)
df_summary = df_summary.reset_index(drop=True)
df_summary['≈pages'] = df_summary['offset'] / 25 + 1
df_summary['Σ%queries'] = df_summary['%queries'].cumsum()
df_summary['%queries'] = df_summary['%queries'].map('{:,.2f}'.format)
df_summary['≈pages'] = df_summary['≈pages'].map('{:,.0f}'.format)
df_summary['Σ%queries'] = df_summary['Σ%queries'].map('{:,.2f}'.format)
i = df_summary.index[-1]
df_summary.at[i, '%queries'] = '≥' + df_summary.at[i, '%queries']
df_summary.at[i, '≈pages'] = '≥' + df_summary.at[i, '≈pages']
df_summary = df_summary.astype({'offset': str})
i = df_summary.index[0]
df_summary.at[i, 'offset'] = 'not specified'
df_summary.to_csv('offset-summary.csv', index=False, float_format='%.2f')
df_summary

,count,offset,%queries,≈pages,Σ%queries
0,11151329,not specified,95.76,0,95.76
1,29953,0,0.26,1,96.02
2,149179,25,1.28,2,97.30
3,63536,50,0.55,3,97.85
4,34860,75,0.30,4,98.15
5,26767,100,0.23,5,98.38
6,16072,125,0.14,6,98.52
7,12021,150,0.10,7,98.62
8,8684,175,0.07,8,98.69
9,8836,200,0.08,9,98.77
